# External Lab 

Here each question is of 1 mark.

# Multiple Linear Regression

## Problem Statement

Use Multiple Linear Regression to **predict the consumption of petrol** given relevant variables are the petrol tax, the per capita, income, the number of miles of paved highway, and the proportion of the population with driver's licenses.

## Dataset

There are 48 rows of data.  The data include:

      I,  the index;
      A1, the petrol tax;
      A2, the per capita income;
      A3, the number of miles of paved highway;
      A4, the proportion of drivers;
      B,  the consumption of petrol.

### Reference 

    Helmut Spaeth,
    Mathematical Algorithms for Linear Regression,
    Academic Press, 1991,
    ISBN 0-12-656460-4.

    S Weisberg,
    Applied Linear Regression,
    New York, 1980, pages 32-33.

## Question 1 - Exploratory Data Analysis

*Read the dataset given in file named **'petrol.csv'**. Check the statistical details of the dataset.*

**Hint:** You can use **df.describe()**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
Petrol = pd.read_csv('petrol.csv', sep=",", header=0)
Petrol.head()

,tax,income,highway,dl,consumption
0,9.0,3571,1976,0.525,541
1,9.0,4092,1250,0.572,524
2,9.0,3865,1586,0.580,561
3,7.5,4870,2351,0.529,414
4,8.0,4399,431,0.544,410


In [3]:
Petrol.dtypes

tax             float64
 income           int64
 highway          int64
 dl             float64
 consumption      int64
dtype: object

In [4]:
Petrol.describe()

,tax,income,highway,dl,consumption
count,48.000000,48.000000,48.000000,48.000000,48.000000
mean,7.668333,4241.833333,5565.416667,0.570333,576.770833
std,0.950770,573.623768,3491.507166,0.055470,111.885816
min,5.000000,3063.000000,431.000000,0.451000,344.000000
25%,7.000000,3739.000000,3110.250000,0.529750,509.500000
50%,7.500000,4298.000000,4735.500000,0.564500,568.500000
75%,8.125000,4578.750000,7156.000000,0.595250,632.750000
max,10.000000,5342.000000,17782.000000,0.724000,968.000000


In [5]:
Petrol.columns = Petrol.columns.str.strip()

# Question 2 - Cap outliers 

Find the outliers and cap them. (Use (Q1 - 1.5 * IQR) as the minimum cap and (Q3 + 1.5 * IQR) as the max cap. The decision criteria is you should consider the datapoints which only falls within this range. The data points which fall outside this range are outliers and the entire row needs to be removed

In [6]:
def MinLevel(x,df):
    IQR = (df[x].quantile(0.75)-df[x].quantile(0.25))
    Min = df[x].quantile(0.25) - 1.5*IQR
    return(Min)

def MaxLevel(x,df):
    IQR = (df[x].quantile(0.75)-df[x].quantile(0.25))
    Max = df[x].quantile(0.75) + 1.5*IQR
    return(Max)

MinLevel('consumption',Petrol)
MaxLevel('highway',Petrol)

13224.625

In [7]:
Petrol = Petrol[(Petrol.tax >= MinLevel('tax',Petrol)) & (Petrol.tax <= MaxLevel('tax',Petrol))]
Petrol = Petrol[(Petrol.income >= MinLevel('income',Petrol)) & (Petrol.income <= MaxLevel('income',Petrol))]
Petrol = Petrol[(Petrol.highway >= MinLevel('highway',Petrol)) & (Petrol.highway <= MaxLevel('highway',Petrol))]
Petrol = Petrol[(Petrol.dl >= MinLevel('dl',Petrol)) & (Petrol.dl <= MaxLevel('dl',Petrol))]
Petrol = Petrol[(Petrol.consumption >= MinLevel('consumption',Petrol)) & (Petrol.consumption <= MaxLevel('consumption',Petrol))]
Petrol.shape

(42, 5)

# Question 3 - Independent variables and collinearity 
Which attributes seems to have stronger association with the dependent variable consumption?

In [8]:
Petrol.corr()

,tax,income,highway,dl,consumption
tax,1.000000,-0.133841,-0.443926,-0.316342,-0.463247
income,-0.133841,1.000000,-0.076862,0.296002,-0.254464
highway,-0.443926,-0.076862,1.000000,0.133983,0.215182
dl,-0.316342,0.296002,0.133983,1.000000,0.549161
consumption,-0.463247,-0.254464,0.215182,0.549161,1.000000


### Observing the above correlation values between all the variables, we can see that there is stronger association between the number of drivers and consumption. And comparatively tax has an association in a negative way. 
Insights :
As tax increases the consumption decreases.
As number of drivers is more consumption is more

# Question 4 - Transform the dataset 
Divide the data into feature(X) and target(Y) sets.

In [9]:
X = Petrol[['tax','dl']]
X.shape

(42, 2)

In [10]:
Y = Petrol['consumption']
Y.shape

(42,)

# Question 5 - Split data into train, test sets 
Divide the data into training and test sets with 80-20 split using scikit-learn. Print the shapes of training and test feature sets.

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=100)

In [12]:
print('X Train',X_train.shape,'\n','X Test',X_test.shape,'\n','Y Train',Y_train.shape,'\n','Y Test',Y_test.shape)

X Train (33, 2) 
 X Test (9, 2) 
 Y Train (33,) 
 Y Test (9,)


# Question 6 - Build Model 
Estimate the coefficients for each input feature. Construct and display a dataframe with coefficients and X.columns as columns

In [13]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
regressor.fit(X_train,Y_train)

regressor.coef_  #.reshape(-1,1)
#regressor.intercept_



array([-25.35163205, 672.00540148])

In [14]:
Coeff = pd.DataFrame(np.transpose(regressor.coef_), X.columns)
Coeff = np.transpose(Coeff)
Coeff

#Did a transpose because the question is asking to use X.columns as Columns.

,tax,dl
0,-25.351632,672.005401


In [15]:
y_pred_test = regressor.predict(X_test)

In [16]:
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

r2_score(Y_test, y_pred_test)

0.4784703649540585

# R-Square 

# Question 7 - Evaluate the model 
Calculate the accuracy score for the above model.

In [17]:
regressor.score(X_test, Y_test)

0.4784703649540585

# Question 8: Repeat the same Multi linear regression modelling by adding both Income and Highway features
Find R2 


In [18]:
X2 = Petrol[['tax','dl','income','highway']]

In [19]:
Y2 = Petrol['consumption']

In [20]:
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=0.2, random_state=100)

In [21]:
print('X2 Train',X2_train.shape,'\n','X2 Test',X2_test.shape,'\n','Y2 Train',Y2_train.shape,'\n','Y2 Test',Y2_test.shape)

X2 Train (33, 4) 
 X2 Test (9, 4) 
 Y2 Train (33,) 
 Y2 Test (9,)


In [22]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
regressor.fit(X2_train,Y2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [23]:
y2_pred = regressor.predict(X2_test)

from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

r2_score(Y2_test, y2_pred)

0.42968756479315084

In [24]:
regressor.score(X2_test, Y2_test)

0.4296875647931508

# Question 9: Print the coefficients of the multilinear regression model

In [25]:
Coeff = pd.DataFrame(np.transpose(regressor.coef_), X2.columns)
Coeff = np.transpose(Coeff)
Coeff

#Did a transpose because the question is asking to use X.columns as Columns.

,tax,dl,income,highway
0,-17.123436,770.633783,-0.071204,0.004898


# Question 10 
In one or two sentences give reasoning on R-Square on the basis of above findings
Answer

### *R squared value increase if we increase the number of independent variables to our analysis

R squared value will increase with increase in number of variables but we use adjusted r square to negate that and get a better sense of model performance.